In [6]:
from flask import Flask, request, jsonify
from flask_cors import CORS  # Import CORS from flask_cors
import cv2
import numpy as np
import dlib
from keras.models import load_model
from keras.preprocessing.image import img_to_array
from threading import Thread
from playsound import playsound
from twilio.rest import Client
import pygame
app = Flask(__name__)

# Enable CORS for all routes
CORS(app)


# Load models and resources (update paths as needed)
face_cascade = cv2.CascadeClassifier(r"C:\Users\Abish\Downloads\data\haarcascade_frontalface_default.xml")
predictor = dlib.shape_predictor(r"C:\Users\Abish\OneDrive\Desktop\Final year project\shape_predictor_68_face_landmarks.dat")
detector = dlib.get_frontal_face_detector()
model = load_model("drowiness_cnn.h5")

# Global state
count = 0
alarm_on = False
last_notification_count = -1  # Track last count when notification was sent



def start_alarm(sound):
    """Play the alarm sound using pygame"""
    pygame.mixer.init()
    pygame.mixer.music.load(sound)
    pygame.mixer.music.play()
    
def stop_alarm():
    """Stop the alarm sound if it's playing"""
    if pygame.mixer.get_init() and pygame.mixer.music.get_busy():
        pygame.mixer.music.stop()

alarm_sound = r"C:\Users\Abish\Downloads\80s-alarm-clock-sound\80s-alarm-clock-sound.mp3"
#start_alarm(alarm_sound)

account_sid = 'AC05793d0b5b8c4aa11b621a72e09f37bf'
auth_token = '4bffb511b9505b8b41772deb9c4b3432'
client = Client(account_sid, auth_token)

'''message = client.messages.create(
  from_='whatsapp:+14155238886',
  body='Drowsiness detected. Stay alert!!!',
  to='whatsapp:+919498475799'
)'''

#print(message.sid)

def is_yawning(landmarks):
    top_lip = landmarks[62]
    bottom_lip = landmarks[66]
    lip_distance = abs(top_lip[1] - bottom_lip[1])
    return lip_distance > 10


@app.route('/process_frame', methods=['POST'])
def process_frame():
    global count, alarm_on, last_notification_count

    # Receive frame
    if 'frame' not in request.files:
        return jsonify({"error": "No frame provided"}), 400
    frame_data = request.files['frame'].read()
    np_frame = np.frombuffer(frame_data, np.uint8)
    frame = cv2.imdecode(np_frame, cv2.IMREAD_COLOR)

    if frame is None:
        return jsonify({"error": "Invalid frame"}), 400

    # Process frame
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = detector(gray)
    result = {"status": "No face detected", "count": count, "alert": False, "notification_sent": False}

    for face in faces:
        landmarks = predictor(gray, face)
        landmarks = np.array([[p.x, p.y] for p in landmarks.parts()])

        left_eye_region = landmarks[36:42]
        right_eye_region = landmarks[42:48]
        left_eye_center = np.mean(left_eye_region, axis=0).astype(int)
        right_eye_center = np.mean(right_eye_region, axis=0).astype(int)

        left_eye_roi = frame[left_eye_center[1] - 30:left_eye_center[1] + 30, left_eye_center[0] - 30:left_eye_center[0] + 30]
        right_eye_roi = frame[right_eye_center[1] - 30:right_eye_center[1] + 30, right_eye_center[0] - 30:right_eye_center[0] + 30]

        if left_eye_roi.size > 0 and right_eye_roi.size > 0:
            left_eye_roi = cv2.resize(left_eye_roi, (145, 145)).astype('float') / 255.0
            right_eye_roi = cv2.resize(right_eye_roi, (145, 145)).astype('float') / 255.0
            left_eye_roi = img_to_array(left_eye_roi)
            right_eye_roi = img_to_array(right_eye_roi)
            left_eye_roi = np.expand_dims(left_eye_roi, axis=0)
            right_eye_roi = np.expand_dims(right_eye_roi, axis=0)

            pred1 = model.predict(left_eye_roi)
            pred2 = model.predict(right_eye_roi)
            status1 = np.argmax(pred1)
            status2 = np.argmax(pred2)
            confidence = {
                "left_eye": float(np.max(pred1) * 100),
                "right_eye": float(np.max(pred2) * 100)
            }

            eye_closed = (status1 == 2 and status2 == 2)
            mouth_open = is_yawning(landmarks)

            if eye_closed or mouth_open:
                count += 1
                result["count"] = count
                result["confidence"] = confidence
                if eye_closed and mouth_open:
                    result["status"] = f"Eyes Closed and Yawning, Frame count: {count}"
                elif eye_closed:
                    result["status"] = f"Eyes Closed, Frame count: {count}"
                else:
                    result["status"] = f"Yawning, Frame count: {count}"
                
                global whatsapp_alert_sent
                global sms_alert_sent
                if count >= 5:
                    result["alert"] = True
                    if not alarm_on:
                        alarm_on = True
                        Thread(target=start_alarm, args=(alarm_sound,), daemon=True).start()
                    if not whatsapp_alert_sent:                   
                        try:
                            message = client.messages.create(
                            from_='whatsapp:+14155238886',  # Twilio Sandbox WhatsApp number
                            body=f"Drowsiness Alert! {result['status']}",
                            to='whatsapp:+919498475799'
                        )
                            print(f"WhatsApp alert sent. SID: {message.sid}")
                            result["notification_sent"] = True
                            whatsapp_alert_sent = True
                        except Exception as e:
                            print(f"Error sending WhatsApp message: {e}")
                            result["notification_sent"] = False
                    if not sms_alert_sent:
                        try:
                            sms = client.messages.create(
                            body=f"Drowsiness Alert! {result['status']}",
                            from_='+19565865752',  
                            to='+919498475799'     
                        )
                            print(f"SMS alert sent. SID: {sms.sid}")
                            sms_alert_sent = True
                        except Exception as e:
                            print(f"Error sending SMS: {e}")

                           
            else:
                result["status"] = "Eyes Open and No Yawn"
                result["confidence"] = confidence
                count = 0
                alarm_on = False
                last_notification_count = -1
                whatsapp_alert_sent = False
                sms_alert_sent = False
                stop_alarm()  

    return jsonify(result)

if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5000, debug=True, use_reloader=False)


 * Serving Flask app '__main__'
 * Debug mode: on


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://192.168.31.45:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [26/May/2025 14:53:20] "OPTIONS /process_frame HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [26/May/2025 14:53:20] "POST /process_frame HTTP/1.1" 200 -


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 208ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 89ms/step


INFO:werkzeug:127.0.0.1 - - [26/May/2025 14:53:21] "POST /process_frame HTTP/1.1" 200 -


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 113ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 98ms/step


...

INFO:werkzeug:127.0.0.1 - - [26/May/2025 14:55:24] "POST /process_frame HTTP/1.1" 200 -


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 102ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 98ms/step


INFO:werkzeug:127.0.0.1 - - [26/May/2025 14:55:25] "POST /process_frame HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [26/May/2025 14:55:25] "OPTIONS /process_frame HTTP/1.1" 200 -


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 106ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 97ms/step


INFO:werkzeug:127.0.0.1 - - [26/May/2025 14:55:26] "POST /process_frame HTTP/1.1" 200 -
